In [9]:
import openslide
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import ast
import tensorflow as tf
from tensorflow import keras

In [10]:
samples = pd.read_csv('../input/mayo-clinic-strip-ai/test.csv')

In [11]:
def get_patch1(img, tl_pixel, patch_shape):
    img = np.array(img.read_region(tl_pixel, 0, patch_shape).convert("RGB"))
    return img

final_samples= pd.DataFrame()
ids = []
for n in tqdm(range(samples.shape[0])):
    img_path = "../input/mayo-clinic-strip-ai/test/"+samples.iloc[n]['image_id']+".tif"
   #label = samples.iloc[n]['label']
    i_id = samples.iloc[n]['image_id']
    ids.append(i_id)
    img  = openslide.open_slide(img_path)
    series = pd.Series()
    n = 0
    for i in range(img.dimensions[0]//25,img.dimensions[0]-img.dimensions[0]//25,512):
        for j in range(img.dimensions[0]//25,img.dimensions[0]-img.dimensions[0]//25,512):
            if n ==20:
                break           
            im = get_patch1(img , (i,j), (512,512) )
            if im.std()>60.0:
                n=n+1
                s = pd.Series(data = [(i,j)])
                series = series.append(s,ignore_index=True)
                series = series.reset_index()
                series = series.drop("index",axis=1)
    if len(series)==0:
        for i in range(img.dimensions[0]//25,img.dimensions[0]-img.dimensions[0]//25,512):
            for j in range(img.dimensions[0]//25,img.dimensions[0]-img.dimensions[0]//25,512):
                if n ==20:
                    break           
                im = get_patch1(img , (i,j), (512,512) )
                if im.std()>50.0:
                    n=n+1
                    s = pd.Series(data = [(i,j)])
                    series = series.append(s,ignore_index=True)
                    series = series.reset_index()
                    series = series.drop("index",axis=1)
    if len(series)==0:
        for i in range(img.dimensions[0]//25,img.dimensions[0]-img.dimensions[0]//25,512):
            for j in range(img.dimensions[0]//25,img.dimensions[0]-img.dimensions[0]//25,512):
                if n ==20:
                    break           
                im = get_patch1(img , (i,j), (512,512) )
                if im.std()>40.0:
                    n=n+1
                    s = pd.Series(data = [(i,j)])
                    series = series.append(s,ignore_index=True)
                    series = series.reset_index()
                    series = series.drop("index",axis=1)
    if len(series)==0:
        for i in range(img.dimensions[0]//25,img.dimensions[0]-img.dimensions[0]//25,512):
            for j in range(img.dimensions[0]//25,img.dimensions[0]-img.dimensions[0]//25,512):
                if n ==20:
                    break           
                im = get_patch1(img , (i,j), (512,512) )
                if im.std()>30.0:
                    n=n+1
                    s = pd.Series(data = [(i,j)])
                    series = series.append(s,ignore_index=True)
                    series = series.reset_index()
                    series = series.drop("index",axis=1)
    final_samples = pd.concat([final_samples , series],axis=1)
    final_samples.columns = ids
final_samples = final_samples.T

  0%|          | 0/4 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  del sys.path[0]
100%|██████████| 4/4 [00:19<00:00,  4.80s/it]


In [12]:
def get_patch(path, tl_pixel, patch_shape):
    img = openslide.open_slide(path)
    img = np.array(img.read_region(tl_pixel, 0, patch_shape).convert("RGB"))
    return cv2.resize(img/255.0, (512, 512))

In [13]:
def sample_gen(df = samples , data = final_samples):
    for i in range(data.shape[0]):
        num_images = 20 - data.iloc[i].isnull().sum()
        images_list = np.arange(num_images)
        if  num_images>=5:
            final_image_list = np.random.choice(images_list, size=(5), replace=False)
            img1 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][final_image_list[0]]),(512,512))
            img2 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][final_image_list[1]]),(512,512))
            img3 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][final_image_list[2]]),(512,512))
            img4 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][final_image_list[3]]),(512,512))
            img5 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][final_image_list[4]]),(512,512))
        else:
            final_image_list = np.random.choice(images_list, size=(5),replace= True)
            img1 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][final_image_list[0]]),(512,512))
            img2 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][final_image_list[1]]),(512,512))
            img3 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][final_image_list[2]]),(512,512))
            img4 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][final_image_list[3]]),(512,512))
            img5 = get_patch('../input/mayo-clinic-strip-ai/test/'+df.iloc[i]['image_id']+'.tif' , (data.loc[df.iloc[i]['image_id']][final_image_list[4]]),(512,512))
        yield ({'input_71':img1,'input_72':img2,'input_73':img3,'input_74':img4,'input_75':img5})
sampleset = tf.data.Dataset.from_generator(
     sample_gen,
     ({'input_71':tf.float32,'input_72':tf.float32,'input_73':tf.float32,'input_74':tf.float32,'input_75':tf.float32}),
    ({'input_71':tf.TensorShape([512,512,3]),'input_72':tf.TensorShape([512,512,3]),'input_73':tf.TensorShape([512,512,3]),'input_74':tf.TensorShape([512,512,3]),'input_75':tf.TensorShape([512,512,3])}))
sampleset = sampleset.batch(1)

In [14]:
model = tf.keras.models.load_model('../input/mayo-patho/best_model/model')

In [15]:

pred = model.predict(sampleset)
result = pd.DataFrame({'patient_id':samples['patient_id'],'CE': pred[:,0],'LAA': 1-pred[:,0]}).groupby("patient_id").mean()
submission = result[["CE", "LAA"]].round(6).reset_index()
submission.to_csv('./submission.csv', index=False)
